## Creating reservoir operations

### USER INPUT

In [1]:
# This is the NetCDF created
created_file = 'Outputs/reservoirOperations_Ebro.nc'

# This is the NetCDF used as the base/template
base = r'P:\watmodel\CWATM\Regions\CWatM-Ebro-1km\input_netcdf\routing\lakereservoirs\lakeresArea.nc'

# Reservoir location indices
# from 0 index, for example, (116, 41) is (117, 42) in panoply
Reservoir_indices = [
    (247, 353),
    (353, 247),
    (248, 354),
    (354, 248),
    (249, 355),
    (355, 249)]

# Fraction of live storage released into channel downstream
# 366-days, for each reservoir
Downstream_release = [
    [0.08]*366,
    [0.08]*366,
    [0.08]*366,
    [0.08]*366,
    [0.08]*366,
    [0.08]*366] #, #[0.01, 0.02, 0.02, ..., 0.03] #366-element array

# Maximum fraction of live storage released to satisfy reservoir/command area demands
# 366-days, for each reservoir
Satisfying_demands = [
    [0.08]*366,
    [0.08]*366,
    [0.08]*366,
    [0.08]*366,
    [0.08]*366,
    [0.08]*366]

### *SCRIPTS*

In [2]:
import netCDF4 as nc4
import numpy as np
import datetime

In [3]:
f2 = nc4.Dataset(created_file,'w', format='NETCDF4')
nc = nc4.Dataset(base,'r')

In [4]:
if 'x' and 'y' in nc.variables.keys():
    xy_dimensions = True
elif 'lon' and 'lat' in nc.variables.keys():
    lonlat_dimensions = True
else:
    exit()

In [5]:
if xy_dimensions:
    f2.createDimension('x', nc.variables['x'].shape[0])
    f2.createDimension('y', nc.variables['y'].shape[0])
elif lonlat_dimensions:
    f2.createDimension('lon', nc.variables['lon'].shape[0])
    f2.createDimension('lat', nc.variables['lat'].shape[0])
    
f2.createDimension('time', 366)

<class 'netCDF4._netCDF4.Dimension'>: name = 'time', size = 366

In [6]:
 time = f2.createVariable('Time', 'i4', 'time')
if xy_dimensions:
    longitude = f2.createVariable('X', 'f4', 'x')
    latitude = f2.createVariable('Y', 'f4', 'y')  
    frac_release = f2.createVariable('Downstream release', 'f4', ('time', 'y', 'x'),zlib=True,least_significant_digit=4)
    frac_demand = f2.createVariable('Fraction of Volume', 'f4', ('time', 'y', 'x'),zlib=True,least_significant_digit=4)

elif lonlat_dimensions:
    longitude = f2.createVariable('Longitude', 'f4', 'lon')
    latitude = f2.createVariable('Latitude', 'f4', 'lat') 
    frac_release = f2.createVariable('Downstream release', 'f4', ('time', 'lat', 'lon'),zlib=True,least_significant_digit=4)
    frac_demand = f2.createVariable('Fraction of Volume', 'f4', ('time', 'lat', 'lon'),zlib=True,least_significant_digit=4)

In [7]:
if xy_dimensions:
    longitude[:] = nc.variables['x'][:] 
    latitude[:] = nc.variables['y'][:]

elif lonlat_dimensions:
    longitude[:] = nc.variables['lon'][:]
    latitude[:] = nc.variables['lat'][:]
    
time[:] = range(366)[:]

for i in range(len(Reservoir_indices)):
    frac_release[:, Reservoir_indices[i][0], Reservoir_indices[i][1]] = Downstream_release[i]
    frac_demand[:, Reservoir_indices[i][0], Reservoir_indices[i][1]] = Satisfying_demands[i]

In [8]:
f2.close()